In [2]:
# Custom Imports
from CircuitCollection import CircuitCollection as cc

# Qiskit Imports
from qiskit import *
from qiskit.transpiler import CouplingMap
from qiskit.tools.jupyter import *
from qiskit.visualization import *

# Other Imports
import itertools
import time
import pickle
import pandas as pd
import numpy as np

## Data Representation
Other works (like [this](https://iopscience.iop.org/article/10.1088/2632-2153/ac28dd)) use an "image" based representation. I will use a similar one.
* each gate will be one column of "pixels"
* 1 indicates that the corresponding qubit is involved with the gate, 0 means that there are no gates on that qubit           

#### Creating Circuit Collection

In [ ]:
#####################
# Run this to generate new circuit collection
#####################
filename = "./data/4_qubits_2_depth_10_trials.p" # file to save circuit collection to

# start timer
start_time = time.time()

# Initialize a 4 qubit circuit
n = 4
depth = 2
trials = 10
gates = [(0, 1), (1, 2), (2, 3), (0, 2), (1, 3), (0, 3)] # full gateset of cnots

# circuit collection
circol = cc(gates, n, depth)

# generate all children
circol.generate_circuits()

# build all circuits
circol.build_circuits()

# transpile all circuits
circol.transpile_circuits(n = 20, trials = trials, coupling_map = CouplingMap.from_line(n), optimization_level=1)

# print run time
print("--- Total: %s seconds ---" % (time.time() - start_time))

# print circuits per second
print("--- %s circuits per second ---" % (circol.num_circuits() * trials/(time.time() - start_time)))

pickle.dump(circol, open(filename, "wb"))

In [3]:
#####################
# Run this to load a previously generated circuit collection
#####################
filename = "../../qcircml_code/data/circol_test.p" # file to load circuit collection 

# load circuit collection
circol = pickle.load(open(filename, "rb"))

# retrieve parameters
n = circol.num_qubits
depth = circol.depth

# print parameters
print("Circuit Collection Paramters:")
print("n = " + str(n))
print("depth = " + str(depth))

Circuit Collection Paramters:
n = 4
depth = 2


#### Computing Best Cuts

In [25]:
# Compute best cut for each circuit

optimal_circuits = []
optimal_cuts = []

for j in range(len(circol.circuits[-1])): # loop through max lenght circuits
    ind = circol.child_indecies(len(circol.circuits) - 1, j) # compute children indecies
    depths = [circol.q_transpiled[n1][n2].depth() for n1, n2 in ind]
    min = depths[np.argmin(depths)]
    min_indexes = np.where(np.array(depths) == min)[0]

    optimal_circuits.append([ind[i] for i in min_indexes]) # choose child with lowest depth

    # compute the index of the cut gate
    parent_gates = circol.circuits[-1][j]
    child_gates = [circol.circuits[optimal_circuits[-1][i][0]][optimal_circuits[-1][i][1]] for i in range(len(optimal_circuits[-1]))]

    temp = []
    for gate in parent_gates:
        b = False

        # check if gate is a best cut
        for child_list in child_gates:
            if gate not in child_list:
                b = True
                break

        if b:
            temp.append(parent_gates.index(gate))
        
    optimal_cuts.append(temp)

# draw a random circuit with the optimal cut
print()
print(optimal_cuts)

test = 8
n1, n2 = optimal_circuits[test]
print(circol.q_transpiled[-1][test].draw())
print(circol.q_transpiled[n1][n2].draw())
print(f"Depth difference: {-circol.q_transpiled[n1][n2].depth() + circol.q_transpiled[-1][test].depth()}")


[5, 6, 7, 6, 4, 6]
[((0, 1), (2, 3), (0, 2), (1, 3), (0, 3))]
[6, 5, 7, 6, 5, 7]
[((0, 1), (1, 2), (2, 3), (0, 2), (1, 3)), ((0, 1), (2, 3), (0, 2), (0, 3), (1, 3))]
[5, 7, 6, 6, 4, 6]
[((0, 1), (2, 3), (1, 3), (0, 2), (0, 3))]
[7, 5, 6, 7, 5, 6]
[((0, 1), (1, 2), (2, 3), (1, 3), (0, 2)), ((0, 1), (2, 3), (1, 3), (0, 3), (0, 2))]
[6, 7, 5, 4, 4, 5]
[((0, 1), (1, 2), (0, 3), (0, 2), (1, 3)), ((0, 1), (2, 3), (0, 3), (0, 2), (1, 3))]
[7, 6, 5, 4, 4, 5]
[((0, 1), (1, 2), (0, 3), (1, 3), (0, 2)), ((0, 1), (2, 3), (0, 3), (1, 3), (0, 2))]
[6, 7, 6, 7, 7, 5]

[[1], [1, 4], [1], [1, 4], [1, 2], [1, 2]]


IndexError: list index out of range

<div class = "alert alert-danger"> FIXME: add some statistics </div>

<div class = "alert alert-danger"> FIXME: https://ekamperi.github.io/machine%20learning/2021/01/13/python-decorators-and-tf-function.html </div>



#### Explanation of Actor-Critic 
* https://www.tensorflow.org/tutorials/reinforcement_learning/actor_critic
* https://arshren.medium.com/unlocking-the-secrets-of-actor-critic-reinforcement-learning-a-beginners-guide-3c5953b13551

In [ ]:
# load circuit collection
circol = pickle.load(open("../qcircml_code/data/circol_test.p", "rb"))

# generate images
circol.convert_to_images()

circol = pickle.dump(circol, open("../qcircml_code/data/circol_test.p", "wb"))

For some reason adding the tf.function decorator causes the model to not converge (does no better than random)
* this is likely due to the face that the tf.function generator compiles the code into a graph which does not allow for the random number generator to be used properly
* it could also be due to some parameters not changing properly during training

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ../qcircml_code/logs/ --host=127.0.0.1